In [132]:

import numpy as np
import matplotlib.pyplot as plt
# A library for data manipulation and analysis.
import pandas as pd
from sklearn.cluster import KMeans
# Output of plotting commands is displayed inline within the Jupyter notebook.
import seaborn as sns

# # Set a seed so that the results are consistent.
# np.random.seed(3) 

## CLEANING DATASET

In [ ]:
df = pd.read_csv('scraped_data_only_gianha_2_new.csv',encoding='utf-8')
df.rename(columns={'Diện tích':'area', 
                   'Mức giá' : 'price',
                   'Số phòng ngủ': 'num_of_bedrooms',
                   'Số toilet': 'num_of_toilets',
                   'Pháp lý': 'legal_status',
                   'Nội thất': 'furniture',
                   'Latitude':'latitude',
                   'Longitude':'longitude',
                   'Mặt tiền': 'facade_width',
                   'Đường vào': 'access_road_width',
                   'Hướng nhà': 'house_direction', 
                   'Hướng ban công': 'balcony_direction', 
                   'Số tầng': 'num_of_stories'},
                   inplace=True)
df

In [134]:
columns_to_drop = ['legal_status', 'furniture', 'facade_width', 'access_road_width', 'house_direction', 'balcony_direction','latitude','longitude']

In [ ]:
df = df.drop(columns=columns_to_drop)
df

In [136]:
def clean_and_calculate_price(row):
    price = row['price']
    area = row['area']
    
    if not price or not area:
        return None
    
    price = price.lower()
    
    if 'thỏa thuận' in price:
        return None
    
    if 'triệu/m²' in price:
        try:
            price_per_m2 = float(price.replace('triệu/m²', '').strip().replace(',', '.'))
            area_value = float(area.replace('m²', '').strip().replace(',', '.'))
            total_price = (price_per_m2 * area_value) / 1000  # Convert to 'tỷ'
            return total_price
        except ValueError:
            return None
    
    try:
        if 'triệu' in price:
            return float(price.replace('triệu', '').replace(',', '.').strip()) / 1000
        elif 'tỷ' in price:
            return float(price.replace('tỷ', '').replace(',', '.').strip())
        else:
            return None
    except ValueError:
        return None

In [ ]:
df['price'] = df.apply(clean_and_calculate_price, axis=1)
df.head(10)

In [138]:

def clean_area(area):
    if isinstance(area,str):
        return float(area.lower().replace('m²','').replace('.','').replace(',','.').strip())
    return area

def clean_rooms(value):
    if isinstance(value,str):
        return float(value.lower().replace('phòng','').strip())
    return value

def clean_stories(value):
    if isinstance(value, str):
        if '5+' in value:
            return None 
        value = value.lower().replace('tầng', '').strip() 
        return float(value)
    return value

In [ ]:
df['area'] = df['area'].apply(clean_area)
df.head(10)


In [ ]:
df['num_of_bedrooms'] = df['num_of_bedrooms'].apply(clean_rooms)
df['num_of_toilets'] = df['num_of_toilets'].apply(clean_rooms)
df['num_of_stories'] = df['num_of_stories'].apply(clean_stories)

df.head(10)

In [ ]:
df.count()

In [142]:
def remove_outlier_usingIQR(df,column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [143]:
cleaned_df = df.copy()

In [ ]:
columns_to_clean = ['area','price','num_of_stories','num_of_bedrooms','num_of_toilets']
for column in columns_to_clean:
    cleaned_df = remove_outlier_usingIQR(cleaned_df, column)
cleaned_df.describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area', y='price', data=df, alpha=0.6, size='price', sizes=(20, 200), hue='price', palette='viridis')
plt.title('Area vs. Price')
plt.xlabel('Area (m²)')
plt.ylabel('Price (tỷ)')
plt.grid(True)
plt.legend(title='Price (tỷ)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
thresholds = [
    (100, 2.5), 
    (175, 4.0), 
]

for area_threshold, price_threshold in thresholds:
    cleaned_df = cleaned_df[~((cleaned_df['area'] > area_threshold) & (cleaned_df['price'] < price_threshold))]

cleaned_df.describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area', y='price', data=cleaned_df, alpha=0.6, size='price', sizes=(20, 200), hue='price', palette='viridis')
plt.title('Area vs. Price')
plt.xlabel('Area (m²)')
plt.ylabel('Price (tỷ)')
plt.grid(True)
plt.legend(title='Price (tỷ)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

## CREATING DATABASE FOR CLEANED DATA

In [159]:
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="2408",
    database="housing_data"
    )
cursor = db.cursor()

In [165]:
create_table_query = """
CREATE TABLE IF NOT EXISTS housing_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    area FLOAT,
    price FLOAT,
    num_of_bedrooms INT,
    num_of_toilets INT,
    num_of_stories INT
)
"""
cursor.execute(create_table_query)

In [173]:
sql = """INSERT INTO `housing_data` (`area`, `price`, `num_of_bedrooms`, `num_of_toilets`, `num_of_stories`)
         VALUES (%s, %s, %s, %s, %s)"""

for index, row in cleaned_df.iterrows():
    cursor.execute(sql, (row['area'], row['price'], row['num_of_bedrooms'], row['num_of_toilets'], row['num_of_stories']))

db.commit()

cursor.close()
db.close()